In [5]:
# Aqui será feito extração de features, no caso, em texto. Ou seja, a partir de algo que talvez não tenha atributos
# tão claros, estou extraindo características que fazem sentido na hora de calcular o modelo.

import pandas as pd
df = pd.read_csv('spam.csv', encoding="ISO-8859-1")

In [6]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df = df.rename(columns={"v1" : "label", "v2":"message"})
df = df.drop(columns = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"])

In [ ]:
df.head()

In [ ]:
#Verificando a qtd de nulos
df.isnull().sum()

In [ ]:
# saber quantas msg sáo classificadas como spam ou náo
df["label"].value_counts()

In [ ]:
#separando a variável dependente(y) da independente(x)
from sklearn.model_selection import train_test_split

X = df["message"]
y = df["label"]

In [ ]:
#settando o test para 20% das observações
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=52)

In [ ]:
#entrando agora na extração de feature
#Já que texto não é número, para a extração da feature, usaremos a bag of words, em que vou transformar caracteres
# e palavras em matrizes. Daí, o uso da CountVectorizer() servirá para vetorizar essas palavras, transformando tudo
# numa matriz. (aqui poderia ser usado o one hot encoding para transformar em 0 e 1 também. Usar tambem a lib nltk, a
# remoção de stopwords). Aqui será utilizada uma maneira muito simples.
from sklearn.feature_extraction.text import CountVectorizer

#instancia a classe
count_vector = CountVectorizer()

# Aqui eu fito a transformada em cima dos meus dados de treino
X_train_counts = count_vector.fit_transform(X_train)
X_train_counts.shape

In [ ]:
print(X_train_counts)

In [ ]:
# Daí, será utilizada duas métricas em conjunto, que é a TF-IDF. Com isso darei um score de importância para
# aquela palavra naquele texto, para podermos classificar.
# TF - frequência de aparição de um termo
# IDF - Inverso da frequência que um termo aparece em diferentes "corpus"
# TF-IDF : term freq * inverse document freq --> classifica a importância

from sklearn.feature_extraction.text import TfidfTransformer

# instancia a classe 
tfidf_transform = TfidfTransformer()

# Ou seja, esse tf-idf vai ser feito em cima dos dados X_train_counts, em que foi feita a bag of words

X_train_tfidf = tfidf_transform.fit_transform(X_train_counts)


In [ ]:
#Com isso, temos a nossa métrica de importância
print(X_train_tfidf)

In [ ]:
# Agora, iniciando o nosso modelo chamado Support Vector Machine. Mas obviamente poderiam ser usados outros modelos, como
# o Random Forest, Logistic Regression, etc.
from sklearn.svm import LinearSVC

#instancia a classe
c = LinearSVC()

c.fit(X_train_tfidf, y_train)

In [ ]:
# Mas antes de prosseguir, podemos fazer de outro jeito. Ao inves de usar o CountVectorizer e depois o TfidfTransformer,
# ou seja, pegar primeiro e fazer a bag of words, depois pegar aquele termo com a frequencia e depois multiplicar
# pelo seu inverso para achar um score, eu posso já usar uma classe que faz isso tudo de uma vez, que é o
# TfidfVectorizer. É importante saber que o passo a passo ocorre com a feature extraction através da bagofwords e depois
# a classificação de importancia.
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

#cria o pipeline com o vetorizador e o modelo
text_c = Pipeline([("tfidf", TfidfVectorizer()), ("c", LinearSVC())])

# diferente do input anterior, com o TfidfVectorizer, eu vou fitar apenas o X_train e não mais aquele
# trabalhado (X_train_tfidf)
# Então, esse é o momento em que eu crio o modelo, somente com as variáveis de treino
text_c.fit(X_train, y_train)

# Aqui, eu aplica o modelo criado nas variáveis X_test, em que ele vai me dizer a previsão de Y para esses X de teste.
predictions = text_c.predict(X_test)

predictions

In [ ]:
# agora irei aplicar 3 métricas para medir a eficiência desse modelo
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

#exibir a matriz de confusão (aqui, eu vou pegar a predictions, que foi o resultado da aplicação do modelo nos X de teste,
# e vou comparar com o y de test, pra ver se o que eu previ está de acordo com o que deveria ter sido)
confusion_matrix(y_test, predictions)

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(accuracy_score(y_test, predictions))